In [1]:
import pandas as pd
# import matplotlib.pyplot as plt
import nltk
import time, os, pickle
from nltk.tokenize import word_tokenize, sent_tokenize
from itertools import combinations, permutations
# plt.style.use('ggplot')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# import data

In [3]:
df1 = pd.read_csv(r'../all-the-news/articles1.csv')
df2 = pd.read_csv(r'../all-the-news/articles2.csv')
df3 = pd.read_csv(r'../all-the-news/articles3.csv')
df = df1.append(df2).append(df3)
df.index = range(df.shape[0])
del df1, df2, df3
df.shape

(142570, 10)

In [4]:
df.sample(5)

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
91429,94888,140476,Jesse Owens’ inspiring story drives ‘Race’,New York Post,Kyle Smith,2016-02-18,2016.0,2.0,http://nypost.com/2016/02/18/jesse-owens-inspi...,Hitler didn’t actually snub Jesse Owens at the...
130178,133641,201703,It sure looks like Aetna quit Obamacare becaus...,Vox,Sarah Kliff,2016/8/17,2016.0,8.0,http://www.vox.com/2016/8/17/12515502/aetna-ob...,"There has been a theory, circulating in Washi..."
11574,11574,30310,AG Sessions: ’No Legal Problem’ with Imprisoni...,Breitbart,Edwin Mora,2017-03-09,2017.0,3.0,NaN,Attorney General (AG) Jeff Sessions indicated ...
126141,129604,194609,"U.S. retail sales, industrial output data poin...",Reuters,Lucia Mutikani,2016-12-14,2016.0,12.0,http://www.reuters.com/article/us-usa-economy-...,U. S. retail sales barely rose in November an...
110685,114148,169006,Should We Really Be Keeping Cats And Dogs — An...,NPR,Barbara J. King,2016-05-05,2016.0,5.0,http://www.npr.org/sections/13.7/2016/05/05/47...,Bioethicist Jessica Pierce includes pets — ...


In [5]:
df.publication.unique()

array(['New York Times', 'Breitbart', 'CNN', 'Business Insider',
       'Atlantic', 'Fox News', 'Talking Points Memo', 'Buzzfeed News',
       'National Review', 'New York Post', 'Guardian', 'NPR', 'Reuters',
       'Vox', 'Washington Post'], dtype=object)

# Define bias of each publication based on mediabiasfactcheck.com

In [6]:
# actual labels from MBFC
bias_dict = {
    'New York Times': 'left-center',
    'Breitbart': 'extreme-right',
    'CNN': 'left',
    'Business Insider': 'left-center',
    'Atlantic': 'left-center',
    'Fox News': 'right',
    'Talking Points Memo': 'left',
    'Buzzfeed News': 'left-center',
    'National Review': 'right',
    'New York Post': 'right-center',
    'Guardian': 'left-center',
    'NPR': 'left-center',
    'Reuters': 'neutral',
    'Vox': 'left',
    'Washington Post': 'left-center'
}

In [7]:
# simplified labels
for k,v in bias_dict.items():
    if 'left' in v:
        bias_dict[k] = 'liberal'
    elif 'right' in v:
        bias_dict[k] = 'conservative'
    else:
        bias_dict[k] = 'neutral'
for k,v in bias_dict.items():
    print(k,v)

New York Times liberal
Breitbart conservative
CNN liberal
Business Insider liberal
Atlantic liberal
Fox News conservative
Talking Points Memo liberal
Buzzfeed News liberal
National Review conservative
New York Post conservative
Guardian liberal
NPR liberal
Reuters neutral
Vox liberal
Washington Post liberal


In [8]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [8]:
# load in the bigrams and lexicons
datapath = r'../data'
with open(os.path.join(datapath, 'libCons.pickle'), 'rb') as f:
    libCons = pickle.load(f)
with open(os.path.join(datapath, 'conCons.pickle'), 'rb') as f:
    conCons = pickle.load(f)
with open(os.path.join(datapath, 'libGrams.pickle'), 'rb') as f:
    libGrams = pickle.load(f)
with open(os.path.join(datapath, 'conGrams.pickle'), 'rb') as f:
    conGrams = pickle.load(f)

# testing

In [9]:
def extractContent(row,n=None):
    sentList = []
    label = bias_dict[row.publication]
    try:
        sentences = sent_tokenize(row.content)
    except:
        sentences = []
    sentences.append(row.title)
    for s in sentences:
        try:
            words = word_tokenize(s)
            words = [w.lower() for w in words]
            bigrams = []
            for i in range(len(words) - 1):
                bigrams.append(' '.join((words[i], words[i+1])))
        except:
            words = []
        if label == 'liberal':
            if n:
                lexPresent = set(words).intersection(libCons[:n])
                gramsPresent = set(bigrams).intersection(libGrams[:n])
            else:
                lexPresent = set(words).intersection(libCons)
                gramsPresent = set(bigrams).intersection(libGrams)
            if lexPresent and gramsPresent:
                sentList.append((label, s, lexPresent, gramsPresent))
        elif label == 'conservative':
            if n:
                lexPresent = set(words).intersection(conCons[:n])
                gramsPresent = set(bigrams).intersection(conGrams[:n])
            else:
                lexPresent = set(words).intersection(conCons)
                gramsPresent = set(bigrams).intersection(conGrams)
            if lexPresent and gramsPresent:
                sentList.append((label, s, lexPresent, gramsPresent))
        else:
            lexPresent = set(words).intersection(set(libCons).union(conCons))
            gramsPresent = set(bigrams).intersection(set(libGrams).union(conGrams))
            if not lexPresent and not gramsPresent:
                sentList.append((label, s, lexPresent, gramsPresent))
    return sentList

In [14]:
sample = df.loc[df.publication == 'Breitbart'].sample().iloc[0]
sample

Unnamed: 0                                                 14380
id                                                         33116
title          At UN, Nikki Haley Has Become the Conscience o...
publication                                            Breitbart
author                                            Frances Martel
date                                                  2017-03-14
year                                                        2017
month                                                          3
url                                                          NaN
content        Upon assuming the presidency, many observers e...
Name: 14379, dtype: object

In [12]:
sample = df.sample().iloc[0]

In [18]:
sentList = extractContent(sample)
len(sentList), len(sample.content)

(1, 5223)

In [19]:
for i,s in enumerate(sentList):
    print(i,s)

0 ('conservative', 'In these three key regions  —   Asia, Eastern Europe, and the Middle East  —   Haley has sent a message to the enemies of freedom that her predecessor, Samantha Power, never bothered with: the United States will not allow the UN to be hijacked by allies of Palestinian terror organizations, or allow it to continue ignoring the seemingly endless transgressions of North Korea, or allow the Russian government to flex its diplomatic muscle to silence its critics in the Security Council.', {'terror', 'freedom', 'east'}, {'in these'})


In [20]:
for i,s in enumerate(sent_tokenize(sample.content)):
    print(i,s)

0 Upon assuming the presidency, many observers expressed reservations about what a Trump administration foreign policy would look like.
1 More than any other representative of President Donald Trump’s government, United Nations Ambassador Nikki Haley has answered those critics.
2 [At the UN, Haley has articulated a clear vision of how America would handle itself on the world stage: a stalwart friend to its allies, fearsome foe to its enemies, and consistent voice of reason in a forum often governed by the whims of authoritarian regimes and violent special interests.
3 This week, for example, Haley made clear the Trump administration would stop at nothing to curb dictator Kim  ’s threats in North Korea and uproot Iranian terror proxies from Syria.
4 “We’re not ruling anything out,” Haley told reporters on Wednesday in response to what the United States would do in response to North Korea’s latest missile tests, which Pyongyang openly admitted were in preparation for an attack on U. S. b

In [21]:
# testing
sents = []
for i in range(20):
    sentList = extractContent(df.iloc[i])
    sents.extend(sentList)
df2 = pd.DataFrame(sents, columns=['label','text','lexicons','bigrams'])
df2.shape

(70, 4)

In [22]:
def extractSentences(df,n=None):
    output = []
    for i in range(df.shape[0]):
        if n:
            sentList = extractContent(df.iloc[i],n)
        else:
            sentList = extractContent(df.iloc[i])
        output.extend(sentList)
    df = pd.DataFrame(output)
    df.columns = ['label', 'text', 'lexicons', 'bigrams']
    return df

In [22]:
start = time.time()
df1 = extractSentences(df,50)
print('runtime (min): ', (time.time() - start) / 60)
df1.groupby('label').count()

runtime (min):  20.47254857222239


,text,lexicons,bigrams
label,,,
conservative,5292,5292,5292
liberal,24374,24374,24374
neutral,127393,127394,127394


In [23]:
start = time.time()
df2 = extractSentences(df)
print('runtime (min): ', (time.time() - start) / 60)
df2.groupby('label').count()

runtime (min):  19.408843235174814


,text,lexicons,bigrams
label,,,
conservative,55770,55770,55770
liberal,216875,216875,216875
neutral,127393,127394,127394


In [24]:
with open(r'../data/filteredNews_top50grams.pickle', 'wb') as f:
    pickle.dump(df1, f, pickle.HIGHEST_PROTOCOL)
with open(r'../data/filteredNews_allgrams.pickle', 'wb') as f:
    pickle.dump(df2, f, pickle.HIGHEST_PROTOCOL)

In [2]:
with open(r'../data/filteredNews_top50grams.pickle', 'rb') as f:
    df1 = pickle.load(f)
with open(r'../data/filteredNews_allgrams.pickle', 'rb') as f:
    df2 = pickle.load(f)

EOFError: Ran out of input

In [12]:
!pwd

/home/alex/Documents/MIDS/w266/final_project/w266_final_project


In [17]:
df.to_csv('filtered_sentences.csv')

In [18]:
import pickle

In [26]:
start = time.time()
with open('filtered_sentences.pickle', 'wb') as f:
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)
time.time() - start

0.7255961894989014

In [27]:
start = time.time()
df.to_csv('filtered_sentences.csv')
df3 = pd.read_csv('filtered_sentences.csv')
time.time() - start

4.2353599071502686

In [24]:
with open('filtered_sentences.pickle', 'rb') as f:
    df2 = pickle.load(f)

In [25]:
df2.head()

,label,text
0,liberal,WASHINGTON — Congressional Republicans have...
1,liberal,The incoming Trump administration could choose...
2,liberal,"In another twist, Donald J. Trump’s administra..."
3,liberal,“Given that this pending litigation involves t...
4,liberal,"“Upon taking office, the Trump administration ..."
